# Análise dos Lotes - Lotes para Uso
### Objetivos:

- Identificar lotes que poderiam ser usados para habitações.
    - Variáveis: ocupação, tamanho (>200m2), estado de conservação e dívida (existência e valor)
- Identificar lotes que poderiam ser usados para equipamentos públicos.
    - Variáveis: ocupação, tamanho (obter filtro com patrícia), estado de conservação e dívida (existência e valor)

### TODO
- Identificar lotes elegíveis para melhoria habitacional.
    - Variáveis de Lotes: ocupação, tamanho (<80m2)
    - cruzar com dados de setores censitários (domicilio01, v018-v022 - esgotamento)
    - cruzar com dados de Zona de Interesse Social
    - cruzar com dados de densidade populacional


In [1]:
import os
import sys

#path para a biblioteca do apiModulo. Ajuste de acordo com a necessidade
sys.path.insert(0, os.path.abspath('../'))
from apiModulo.api import *

## Obtendo os lotes da base

In [2]:
lotes = obterCamada('lotes')
lotes.columns

Index(['level_0', 'index', 'setor', 'quadra', 'distrito', 'setor_novo',
       'quadra_nova', 'inscricao_anterior', 'area_semfaz', 'uso_semfa',
       'tipologia_semfaz', 'foto', 'inscricao_nova', 'logradouro', 'bairro',
       'tombamento', 'divida_ipt', 'geometria', 'id', 'cod_lote', 'gabarito',
       'uso_htl', 'tipologia_htl', 'conservacao', 'ocupacao', 'vocacao',
       'numero', 'tp_coleta', 'estilo', 'google_ano', 'AREA_HTL', 'id_pa',
       'paisagem_c', 'un_paisage', 'check_in', 'ha_lot', 'cod_camada',
       'fk_camada'],
      dtype='object')

# Analisando ocupação na base
- Campo "finais_status_ocu"
- Ao todo 2640 lotes classificados dessa maneira

In [4]:
ocupacao = lotes[~lotes['ocupacao'].isnull()]
ocupacao.shape[0]

2640

In [3]:
#adicionando equipamentos informados pela prefeitura
equipamentos = obterCamada('equipamento_publico', simples=False)

In [14]:
import ipywidgets as widgets
import numpy as np

#layout
area_btn = widgets.IntSlider(value=200,min=ocupacao['AREA_HTL'].min(),max=ocupacao['AREA_HTL'].max(),step=10,description='Área Mínima:')
ocupacao_btn = widgets.Dropdown(options=ocupacao['ocupacao'].unique(),value='Edificação sem uso', description='Ocupação:')
conservacao_btn = widgets.Dropdown(options=ocupacao['conservacao'].unique(),value='Péssimo ou em Risco', description='Conservação:')
tipo_btn =  widgets.Dropdown(options=['Polígono', 'Ponto', 'Calor'],value='Polígono', description='Lotes em:')
equipamento_btn =  widgets.Dropdown(options=equipamentos['tipo'].unique(),value='Educação',description='Equipamentos:')
button = widgets.Button(description='Carregar',disabled=False)
grid = widgets.GridBox([ocupacao_btn, conservacao_btn, area_btn, equipamento_btn, tipo_btn, button], layout=widgets.Layout(grid_template_columns="repeat(3, 400px)"))
output = widgets.Output()
display(grid, output)

#ação quando selecionar itens
def on_button_clicked(b):
    with output:
        output.clear_output()        
        filtrado = ocupacao[ (ocupacao['AREA_HTL']>=area_btn.value) & (ocupacao['ocupacao']==ocupacao_btn.value)]
        equip_filtrado = equipamentos[ (equipamentos['tipo'] == equipamento_btn.value) ]
        print(f"Total selecionado: {filtrado.shape[0]}")

        
        m = visMultiMapa(width='90%', MAPA_ZOOM=15)
        if tipo_btn.value == 'Polígono':
            m = visMultiMapa(m, tipo='calor', dado=equip_filtrado, variavel='tipo')        
            m = visMultiMapa(m, tipo='choro', 
                        dado=filtrado,                         
                        variavel='ocupacao',
                        alias=f"Lotes com  Ocupação: {ocupacao_btn.value}, áre mínima: {area_btn.value} e estado de conservação: {conservacao_btn.value}" )   
        else: 
            como_pontos = filtrado
            como_pontos['geometria'] = como_pontos['geometria'].centroid
            if tipo_btn.value == 'Calor':
                m = visMultiMapa(m, tipo='marcador', dado=equip_filtrado, variavel='tipo')        
                m = visMultiMapa(m, tipo='calor', 
                            dado=como_pontos,  
                            variavel='ocupacao',
                            alias=f"Lotes com  Ocupação: {ocupacao_btn.value}, áre mínima: {area_btn.value} e estado de conservação: {conservacao_btn.value}" )   
            else:
                m = visMultiMapa(m, tipo='calor', dado=equip_filtrado, variavel='tipo')        
                m = visMultiMapa(m, tipo='marcador', 
                            dado=como_pontos,  
                            variavel='ocupacao',
                            alias=f"Lotes com  Ocupação: {ocupacao_btn.value}, áre mínima: {area_btn.value} e estado de conservação: {conservacao_btn.value}" )   
        display(m)

button.on_click(on_button_clicked)    

GridBox(children=(Dropdown(description='Ocupação:', index=2, options=('Edificação Subutilizada', 'Estacionamen…

Output()